# Data Playground

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)

## Scraping cointelepgraph

In [3]:
from data_ingestion_pipeline.crawlers.coin_telegraph import CoinTelegraphCrawler

Init data ingestion pipeline /Users/camiloramirezf/Documents/mai-repos/text-fetch-etl/data_ingestion_pipeline
Connection to database with uri: mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set successful


In [ ]:
from selenium import webdriver
from tempfile import mkdtemp
options = webdriver.ChromeOptions()
options.binary_location = "/opt/chrome/chrome"
options.add_argument("--no-sandbox")
options.add_argument("--headless=new")
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-dev-tools")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--no-zygote")
options.add_argument(f"--user-data-dir={mkdtemp()}")
options.add_argument(f"--data-path={mkdtemp()}")
options.add_argument(f"--disk-cache-dir={mkdtemp()}")
options.add_argument("--remote-debugging-port=9222")

scroll_limit = 5

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get("https://cointelegraph.com/")

In [ ]:
# crawler = CoinTelegraphCrawler()
# crawler.extract("")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://cointelegraph.com/")

# Create an ActionChains object
actions = ActionChains(driver)

In [ ]:
main_carousel = driver.find_element(By.XPATH, "//div[@data-testid='carousel-main']")
first_li = main_carousel.find_element(By.XPATH, "//a[@data-testid='main-news-controls__link']")
first_li.click()

In [ ]:
first_li.is_displayed()

True

In [ ]:
from bs4 import BeautifulSoup
import time
scrolls = 0
max_scrolls = 20
last_height = 0

while True:
    # # Scroll down the page a few times
    driver.execute_script("window.scrollBy(0, 2 * window.innerHeight);")

    # Wait to load page
    time.sleep(2)

    # Break the loop if no more new content
    # new_height = driver.execute_script("return document.body.scrollHeight")
    # if new_height == last_height:
    #     break
    # last_height = new_height

    scrolls += 1
    if scrolls == max_scrolls:
        break

In [ ]:
articles = []

# Parse the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all the 'article' elements
article_elements = soup.find_all('article')

# For each 'article', find the 'h1' element and all 'div' siblings
for article in article_elements:
    h1_element = article.find('h1')
    title = h1_element.text
    summary = h1_element.find_next_sibling('div').text
    content_div = article.find('div', class_='post__content-wrapper')
    time_element = article.find('time')
    datetime = time_element.get('datetime') if time_element else None
    ps = content_div.find_all('p')
    content = ' '.join(p.text for p in ps if "Advertisement" not in p.text)

    # Create a dictionary for each article and append it to the list
    articles.append({'title': title, 'summary': summary, 'content': content, 'datetime': datetime})

# Close the driver
# driver.quit()

In [ ]:
import textwrap
print(textwrap.fill(articles[7]['title'], width=100))

 Weekend Wrap: SOL set for bronze, Bitcoiner fat fingers $100K fee and more


In [ ]:
len(articles)

12

## Scrapping Newsbtc.com

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

url = "https://www.newsbtc.com/news/"

driver = webdriver.Chrome()
driver.get(url)

In [7]:
from urllib.parse import urlparse
from data_ingestion_pipeline.models.documents import ArticleDocument

def extract_article(driver, link):
    
    parsed_url = urlparse(link)

    # Split the path to get components
    path_components = parsed_url.path.split('/')

    # Extract the topic and title
    topic = path_components[2]  # 'company'
    title = path_components[-2].replace('-', ' ')
    # Parse the page source
    driver.get(link)
    time.sleep(1) # wait for content to load     
    soup = BeautifulSoup(driver.page_source, "html.parser")
    soup.find('div', class_='entry_content')

    # Find all h1, h2 and p elements within the jeg_inner_content div
    relevant_elements = soup.find('div', class_='content-inner').find_all(['h1', 'h2', 'p'])
    # Extract the text content from each element
    extracted_text = [element.text.strip() for element in relevant_elements]
    text = " ".join(extracted_text)

    # Print the extracted text
    return ArticleDocument(source="newsbtc", title=title, content=text)
    

In [8]:
import time
# Find the list of articles and extract links 
links = []
num_loads = 2

parent = driver.find_element(By.ID, "load-more-article-stream")

for _ in range(num_loads):
    driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
    
    # load more button
    load_button = driver.find_element(By.ID, "load-more-container")
    load_button.click()
    time.sleep(1)

article_elements = parent.find_elements(By.TAG_NAME, 'article')

links = [article.find_element(By.TAG_NAME, 'a').get_attribute('href') for article in article_elements]

In [9]:
# Visit each link saved in links
articles = []
for link in links:
    text = extract_article(driver, link)
    articles.append(text)
    time.sleep(2)  # Adjust the sleep time as needed to allow the page to load

# Close the driver after visiting all links
driver.quit()

In [83]:
import textwrap

print(textwrap.fill(text, width=100))

The overall market is in a bit of a slump, with none of the top 20 cryptocurrencies moving over 3%
in the last 24 hours. However, Pepe (PEPE) has grown over 5% today, and many analysts see this as a
sign PEPE still has a lot of growth left. Let’s delve into some analysts’ forecasts for Pepe as well
as other similar meme coins. Pepe’s price has grown 5.84% today up to $0.00001279 per coin. However,
its daily trading volume is pumping even more up 46.07% all the way to $1,108,137,354. Part of this
growth is due to the recent approval of Ethereum ETFs (exchange-traded funds.)  With institutions
exposing themselves to ETH, tokens on the blockchain naturally tend to gain more interest. With Pepe
being one of ETH’s biggest tokens, investors looking to diversify into lower market cap projects are
stocking up their portfolios. Popular crypto analyst CorgiI with over 28,000 followers on X
(Twitter) noticed this interest earlier in the day, predicting the market to move PEPE past its all-
time h

In [73]:
len(extracted_text)

43

In [74]:
extracted_text[42]

'The cryptocurrency world is no stranger to meme coins, with Dogecoin (DOGE) leading the pack for years. However, a new...'

## Mongo Writing and reading test


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)
from data_ingestion_pipeline.models.documents import ArticleDocument
from data_ingestion_pipeline.crawlers.coin_telegraph import CoinTelegraphCrawler

Connection to database with uri: mongodb://localhost:30001,localhost:30002/?replicaSet=my-replica-set successful


### Write to local mongo replica

Use a test db and a test collection to write


In [ ]:
from pymongo import MongoClient
mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "db_test"
collection_name = "test"
test_data = {"name": "Copy dino"}

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    result = collection.insert_one(test_data)
    print(f"Data inserted with _id: {result.inserted_id}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

INFO:pymongo.serverSelection:{"message": "Waiting for suitable server to become available", "selector": "<function writable_server_selector at 0x107f19ea0>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 663dd4d6c7ab7bcef353bfd2, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 30001) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30002) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30003) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "663dd4d6c7ab7bcef353bfd2"}, "remainingTimeMS": 29}


Data inserted with _id: 663dd4d6c7ab7bcef353bfd3


### Write using crawler and the model classes

In [ ]:
crawler = CoinTelegraphCrawler()
crawler.extract()

In [ ]:
# test saving articles
doc = ArticleDocument(
    source="test",
    title="Moon will show",
    summary="It is happening, everything is mooning",
    content="Moon is the only destiny now",
    published_at="2024-05-05"
)

# articles = [doc]
# crawler.save(articles)

In [ ]:
doc.model_dump(exclude="id")

{'published_at': '2024-05-08',
 'title': 'Moon will show',
 'content': 'Moon is the only destiny now',
 'summary': 'It is happening, everything is mooning',
 'date': '2024-05-05'}

### Reading 


In [ ]:
from pymongo import MongoClient

mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "crypto-articles"
collection_name = "articles"

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    documents = collection.find({})
    for doc in documents:
        print(doc)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

{'_id': '872ef4a5-399f-45e3-b38a-7fd7557670b5', 'published_at': '2024-05-08', 'title': 'Moon soon', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '61f69029-ba68-436e-936c-c5287e67b8a0', 'published_at': '2024-05-08', 'title': 'Moon later', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'b077e14e-0261-4d26-9f05-ce5e8e4c56d1', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'cfe359f5-028f-45f3-bff7-6e30a6cec9be', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '4ceb0d85-35cd-4574-8afa-7a8cd2d038d9', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only d

## Telegram backfill test

In [50]:
from data_ingestion_pipeline.crawlers import TelegramChannelsCrawler
telegram_crawler = TelegramChannelsCrawler()
result_day = await telegram_crawler.extract_day("2024-06-02")
result_total = await telegram_crawler.extract()

INFO:telethon.network.mtprotosender:Connecting to 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.92:443/TcpFull complete!
INFO:crawlers.telegram:Telegram client connected
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.92:443/TcpFull complete!
INFO:telethon.network.mtprotosender:Connecting to 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.92:443/TcpFull complete!
INFO:crawlers.telegram:Telegram client connected
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.92:443/TcpFull complete!


In [51]:
def to_df(result):
    import pandas as pd
    df =  pd.DataFrame.from_records([record.model_dump() for record in result])
    df["parsed_dt"] = pd.to_datetime(df["published_at"])
    return df


In [52]:
day_df =  to_df(result_day)
total_df = to_df(result_total)
day_df.describe()

,parsed_dt
count,6
mean,2024-06-02 08:00:00
min,2024-06-02 00:00:00
25%,2024-06-02 00:00:00
50%,2024-06-02 00:00:00
75%,2024-06-02 18:00:00
max,2024-06-03 00:00:00


In [53]:
day_df.head()

,id,published_at,source,title,content,summary,parsed_dt
0,606e9002-2c5a-4ef9-b330-a8f5bdb68929,2024-06-02,telegram,🚀 Zentry Secures Strategic Funding to Propel M...,🚀 Zentry Secures Strategic Funding to Propel M...,None,2024-06-02
1,bff756a9-04b3-486f-b8d4-0472112180a0,2024-06-02,telegram,🔒 Trump's Legal Woes: A 17% Chance of Prison B...,🔒 Trump's Legal Woes: A 17% Chance of Prison B...,None,2024-06-02
2,a84b9c79-8167-468f-b636-e286d48746e6,2024-06-03,telegram,🤯 MicroStrategy & Michael Saylor Deny Wrongdoi...,🤯 MicroStrategy & Michael Saylor Deny Wrongdoi...,None,2024-06-03
3,98f82e0e-baae-43b1-975f-8abc572520c4,2024-06-03,telegram,#KNINE up over 130% in 24 hours - the beginnin...,#KNINE up over 130% in 24 hours - the beginnin...,None,2024-06-03
4,e0f33d6e-4861-4c18-a00b-9f27feca7fdb,2024-06-02,telegram,🌟 From Facebook's Libra to Sui: A Deep Tech Jo...,🌟 From Facebook's Libra to Sui: A Deep Tech Jo...,None,2024-06-02


In [60]:
import re
rexp = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
total_df["has_links"] = total_df["content"].apply(lambda x: bool(re.search(rexp, x)))
total_df["has_links"].sum()

0

In [55]:
for i, row in total_df[~total_df["has_links"]].iterrows():
    print(f"\n{i}")
    print(row["content"])


0
🚀 Zentry Secures Strategic Funding to Propel Metagame Layer Development

💰 Zentry, formerly known as GuildFi, has just secured a major funding boost in its mission to build the groundbreaking Metagame Layer.

🤝 Investments from Spartan Capital, Synergis, LongHash Ventures, and DWF Labs have poured in, joining the ranks of early supporters like Animoca, Binance Labs, Coinbase Ventures, and Pantera Capital.

🌐 The funding has catapulted Zentry's treasury to an impressive $150 million, including a substantial stash of 16,000+ ETH. This puts the company in a prime position to forge ahead with the development of the Metagame Layer for next-gen web3 gaming and social experiences.

🎮 "We're on a mission to unite all games and experiences into one vibrant Play Economy," says Zentry's visionary founder and CEO Jarindr Thitadilaka. With Spartan Capital, Synergis, and LongHash onboard, Zentry gains invaluable expertise in gaming, virtual worlds, and decentralized tech.

➡️ In April, GuildFi un

## NewsAPI Test

In [56]:
from newsapi import NewsApiClient
from data_ingestion_pipeline.config import settings
from data_ingestion_pipeline.crawlers import NewsFetcher
from data_ingestion_pipeline.models.documents import ArticleDocument
from datetime import datetime, timedelta

date_format="%Y-%m-%d"
start_dt = datetime.now() - timedelta(days=1)
end_dt = datetime.now() - timedelta(days=1)

api_articles = NewsFetcher().extract(
    page=1,
    start_date=start_dt.strftime(date_format),
    end_date=end_dt.strftime(date_format)
)

In [57]:
ArticleDocument.dump_json(api_articles)

In [58]:
import textwrap
filled = textwrap.fill(api_articles[0].content, 150)
print(filled)

President Joe Biden vetoed a resolution late Friday that passed through both the U.S. House and Senate and was widely supported by the cryptocurrency
industry. And its just the latest move from a pre… [+4169 chars]


In [59]:
df["parsed_dt"].describe()

NameError: name 'df' is not defined

In [ ]:
items = [{"id": 1, "color": "red"}, {"id": 1, "color": "black"}]

a, b= zip(*items)

In [ ]:
b

('color', 'color')